***Посмотрим какие жанры наиболее выпускаемые и используем их для сбора данных***

In [ ]:
import requests
import json
import csv
import numpy as np
import pandas as pd

In [1]:
API_key = "d081a22bc7714c7f9d5d381403505669"

url = f"https://api.rawg.io/api/genres"
resp = requests.get(url, params={"key": API_key}, timeout=10)

if resp.status_code != 200:
    print("Ошибка:", resp.status_code)
    print(resp.text[:500])
else:
    data = resp.json()


In [ ]:
d=dict()
for g in data.get("results", []):
    name = g.get("name")
    count = g.get("games_count")
    d[name]= count
    print(f"{name} - {count}")

In [3]:
df = pd.DataFrame.from_dict(d, orient='index', columns=['games_count']).reset_index()
df = df.rename(columns={'index': 'genre'})
df= df.sort_values('games_count', ascending=False).reset_index().drop(columns='index')
df

,genre,games_count
0,Action,190929
1,Adventure,151116
2,Platformer,100905
3,Puzzle,97372
4,Indie,85394
5,Simulation,76455
6,Casual,66992
7,Strategy,62134
8,RPG,61659
9,Shooter,59603


***Посмотрим на то, какие id у платформ чтобы и также по аналогии с первым пунктом сделать выборку по популярной платорме***

In [4]:
import requests
import json
API_key = "d081a22bc7714c7f9d5d381403505669"

url = f"https://api.rawg.io/api/platforms"
resp = requests.get(url, params={"key": API_key}, timeout=10)

if resp.status_code != 200:
    print("Ошибка:", resp.status_code)
    print(resp.text[:500])
else:
    data = resp.json()

In [5]:
rows = []
for p in data.get("results", []):
    rows.append({
        "id": p.get("id"),
        "platform": p.get("name"),
        "games_count": p.get("games_count")
    })

df = pd.DataFrame(rows)
df = df.sort_values("games_count", ascending=False).reset_index(drop=True)
df

,id,platform,games_count
0,4,PC,559070
1,5,macOS,107814
2,6,Linux,80202
3,3,iOS,77418
4,21,Android,52488
5,18,PlayStation 4,6948
6,7,Nintendo Switch,5718
7,1,Xbox One,5713
8,16,PlayStation 3,3165
9,14,Xbox 360,2808


# Набираем данные

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=action&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),action.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            # защититься от None
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1

            # если встретили игру с rating == 0 -> завершаем парсинг, чтобы не захватывать нерелевантные игры
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break

        # перейти к следующей странице, если есть next
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")


In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=adventure&ordering=-rating"
k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),adventure.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")


            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=platformer&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),platformer.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=puzzle&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),puzzle.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")

            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=indie&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),indie.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=simulation&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),simulation.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=strategy&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),strategy.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=shooter&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),shooter.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=racing&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),racing.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=sports&ordering=-rating"
k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),sports.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=arcade&ordering=-rating"
k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),arcade.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=casual&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),casual.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&platforms=187&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице
OUT_CSV = "rawg_games(2010-2025),ps5.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&platforms=1&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),xb1.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")